# Running Mass Inference on RMP Dataset

In [1]:
from transformers import pipeline
import pandas as pd
import numpy as np

Downloading and using the models through the Pipline API. Running three types of NLP tasks:
1. Polarity Score
2. Emotions
3. Summarizations.

Storing them in three seperate csv files:
For polarity score, each course review will receive a score. 
For emotions, each course review will receive a top 3 emotion
For summarizations, each course will get a summarized result of all its comments. 

In [5]:
polarity_score_predictor = pipeline(model="distilbert-base-uncased-finetuned-sst-2-english", task="sentiment-analysis")
emotion_predictor = pipeline(model="SamLowe/roberta-base-go_emotions", task="text-classification", top_k=3)
summarizer = pipeline(model="facebook/bart-large-cnn", task="summarization")



In [2]:
RATING_PATH = "../data/clean_ratings.csv"
PROF_PATH = "../data/clean_prof_info.csv"

rating = pd.read_csv(RATING_PATH)
prof = pd.read_csv(PROF_PATH)

Running sentiment polarity score prediction on all 50k+ course reviews. 

In [ ]:
rating["polarity_score"]=polarity_score_predictor(rating["comment"].tolist())

In [19]:
polarity_dataset = rating[["class", "comment", "polarity_score"]]
polarity_dataset.to_csv("../data/course_review_polarity.csv")